In [1]:
import pypetl
from datetime import datetime, timedelta, date, time

2022-08-24 04:28:01 >  pypetl.core.preference: Loaded!
2022-08-24 04:28:01 >  pypetl.core.aws.getSecretAll(): Starting...
2022-08-24 04:28:01 >     pypetl.core.aws.getSecret(renos-db-rds-staging-data, rds): Starting...
2022-08-24 04:28:01 >        pypetl.core.aws.validateSecretEngine(postgres): Validating...
2022-08-24 04:28:01 >        pypetl.core.aws.validateSecretEngine(postgres): Validated as database!
2022-08-24 04:28:01 >     pypetl.core.aws.getSecret(renos-db-rds-staging-data, rds): Done!
2022-08-24 04:28:01 >     pypetl.core.aws.getSecret(renos-wh-redshift-data, redshift): Starting...
2022-08-24 04:28:02 >        pypetl.core.aws.validateSecretEngine(redshift): Validating...
2022-08-24 04:28:02 >        pypetl.core.aws.validateSecretEngine(redshift): Validated as database!
2022-08-24 04:28:02 >     pypetl.core.aws.getSecret(renos-wh-redshift-data, redshift): Done!
2022-08-24 04:28:02 >     pypetl.core.aws.getSecret(renos-data-bastion, bastion): Starting...
2022-08-24 04:28:02 > 

In [2]:
pypetl.engine.start()

2022-08-24 04:28:03 >  pypetl.engine.start(): Starting...
2022-08-24 04:28:03 >     pypetl.core.connection.startSSHAll(): Starting...
2022-08-24 04:28:03 >        pypetl.core.connection.startSSHFromSecret(bastion): Starting...
2022-08-24 04:28:03 >           pypetl.core.connection.mergeRemoteAddress(['rds', 'redshift']): Merging multiple remote addresses...
2022-08-24 04:28:03 >           pypetl.core.connection.mergeRemoteAddress(['rds', 'redshift']): Merged!
2022-08-24 04:28:03 >           pypetl.core.connection.startSSH(ec2-54-251-208-105.ap-southeast-1.compute.amazonaws.com, 22, [('renos-v2-staging-rds.cssnfgwcjwmm.ap-southeast-1.rds.amazonaws.com', 5432), ('renos-wh.ciju13xeq9ht.ap-southeast-1.redshift.amazonaws.com', 5439)]): Starting...
2022-08-24 04:28:03 >              pypetl.core.connection.generateTunnelForwarder(ec2-54-251-208-105.ap-southeast-1.compute.amazonaws.com, 22, [('renos-v2-staging-rds.cssnfgwcjwmm.ap-southeast-1.rds.amazonaws.com', 5432), ('renos-wh.ciju13xeq9ht.a

In [8]:
pypetl.engine.stop()

2022-08-23 08:13:36 >  pypetl.engine.stop(): Stopping...
2022-08-23 08:13:36 >     pypetl.core.connection.stopDBAll(): Stopping...
2022-08-23 08:13:36 >        pypetl.core.connection.stopDBFromSecret(rds): Stopping...
2022-08-23 08:13:36 >        pypetl.core.connection.stopDBFromSecret(rds): Stopped!
2022-08-23 08:13:36 >        pypetl.core.connection.stopDBFromSecret(redshift): Stopping...
2022-08-23 08:13:36 >        pypetl.core.connection.stopDBFromSecret(redshift): Stopped!
2022-08-23 08:13:36 >     pypetl.core.connection.stopDBAll(): Stopped!
2022-08-23 08:13:36 >     pypetl.core.connection.stopSSHAll(): Stopping...
2022-08-23 08:13:36 >        pypetl.core.connection.stopSSHFromSecret(bastion): Stopping...
2022-08-23 08:13:37 >        pypetl.core.connection.stopSSHFromSecret(bastion): Stopped!
2022-08-23 08:13:37 >     pypetl.core.connection.stopSSHAll(): Stopped!
2022-08-23 08:13:37 >  pypetl.engine.stop(): Stopped!


# For Fact

In [20]:
tablename = 'rns_payment'
select_data = 'payment_id, user_id, payment_channel_id, payment_number, payment_amount, payment_created_date, payment_expired_date, payment_paid_date, payment_api_request, payment_api_response, date_in, user_in, date_up, user_up, status_record, payment_fee'
constraint_pk = 'payment_id'
source_pk = ''
source_date_up = 'date_up'
source_date_in = 'date_in'

tablename_source = tablename
rds = 'public.%s'%(tablename_source)
rds_metadata = ''
redshift = 'renosdb_fact.%s'%(tablename_source)
redshift_metadata = 'renosdb_metadata.%s'%(tablename_source)
fname = "pipeline_fact.%s.run()"%(tablename_source)
select_metadata = 'id, date_in, user_in, referenced_id, referenced_%s, referenced_date_etl'%(constraint_pk)

tablename = redshift
tablename_metadata = redshift_metadata

# For Dimension

In [51]:

tablename = 'rns_status'
select_data = 'status_id, status_name, date_in, user_in, date_up, user_up, status_record'
constraint_pk = 'status_id'
source_pk = ''
source_date_up = 'date_up'
source_date_in = 'date_in'

tablename_source = tablename
rds = 'public.%s'%(tablename_source)
rds_metadata = ''
redshift = 'renosdb_dimension.%s'%(tablename_source)
redshift_metadata = 'renosdb_metadata.%s'%(tablename_source)
fname = "pipeline_dimension.%s.run()"%(tablename_source)
select_metadata = 'id, date_in, user_in, referenced_id, referenced_%s, referenced_date_etl'%(constraint_pk)

tablename = redshift
tablename_metadata = redshift_metadata

# For All

In [54]:

pypetl.executeDBSecret('redshift','CREATE TABLE IF NOT EXISTS %s ( id bigint NOT NULL identity(1,1) );'%(tablename_metadata))
pypetl.executeDBSecret('redshift','ALTER TABLE %s ADD COLUMN date_in timestamp;'%(tablename_metadata))
pypetl.executeDBSecret('redshift','ALTER TABLE %s ADD COLUMN user_in varchar(256);'%(tablename_metadata))
pypetl.executeDBSecret('redshift','ALTER TABLE %s ADD COLUMN referenced_id bigint;'%(tablename_metadata))
pypetl.executeDBSecret('redshift','ALTER TABLE %s ADD COLUMN referenced_%s bigint;'%(tablename_metadata, constraint_pk))
pypetl.executeDBSecret('redshift','ALTER TABLE %s ADD COLUMN referenced_date_etl timestamp;'%(tablename_metadata))

pypetl.executeDBSecret('redshift','CREATE TABLE IF NOT EXISTS %s ( %s bigint NOT NULL );'%(tablename, constraint_pk))
pypetl.executeDBSecret('redshift','ALTER TABLE %s ADD COLUMN status_name varchar(256);'%(tablename))
pypetl.executeDBSecret('redshift','ALTER TABLE %s ADD COLUMN date_in timestamp;'%(tablename))
pypetl.executeDBSecret('redshift','ALTER TABLE %s ADD COLUMN user_in varchar(256);'%(tablename))
pypetl.executeDBSecret('redshift','ALTER TABLE %s ADD COLUMN date_up timestamp;'%(tablename))
pypetl.executeDBSecret('redshift','ALTER TABLE %s ADD COLUMN user_up varchar(256);'%(tablename))
pypetl.executeDBSecret('redshift','ALTER TABLE %s ADD COLUMN status_record varchar(256);'%(tablename))
pypetl.executeDBSecret('redshift','ALTER TABLE %s ADD COLUMN date_etl timestamp;'%(tablename))
pypetl.executeDBSecret('redshift','ALTER TABLE %s ADD COLUMN referenced_id bigint;'%(tablename))

# 'ALTER TABLE %s RENAME COLUMN id to order_status_id'%(tablename)
# 'ALTER TABLE %s ADD PRIMARY KEY (id);'%(tablename)

# 'ALTER TABLE %s ADD COLUMN user_in varchar(256);'%(tablename_metadata)
# 'ALTER TABLE %s ADD COLUMN referenced_id bigint;'%(tablename_metadata)
# 'ALTER TABLE %s ADD COLUMN referenced_%s bigint;'%(tablename_metadata, constraint_pk)
# 'ALTER TABLE %s ADD COLUMN referenced_date_etl timestamp;'%(tablename_metadata)

# 'CREATE TABLE IF NOT EXISTS %s ( %s bigint NOT NULL );'%(tablename, constraint_pk)
# BIGINT
# 'ALTER TABLE %s ADD COLUMN category_parent_id bigint;'%(tablename)
# DECIMAL
# 'ALTER TABLE %s ADD COLUMN product_price decimal;'%(tablename)
# TIMESTAMP
# 'ALTER TABLE %s ADD COLUMN date_up timestamp;'%(tablename)
# VARCHAR
# 'ALTER TABLE %s ADD COLUMN status_record varchar(256);'%(tablename)
# INTEGER
# 'ALTER TABLE %s ADD COLUMN is_active integer;'%(tablename)
# BOOLEAN
# 'ALTER TABLE %s ADD COLUMN is_active boolean;'%(tablename)

# 'ALTER TABLE %s ADD COLUMN date_etl timestamp;'%(tablename)
# 'ALTER TABLE %s ADD COLUMN referenced_id bigint;'%(tablename)

# 'ALTER TABLE %s DROP COLUMN shipping_type_description;'%(tablename)
# 'TRUNCATE %s ;'%(tablename)
# 'TRUNCATE %s ;'%(tablename_metadata)
# 'DROP TABLE %s'%(tablename)

# 'ALTER TABLE %s ADD COLUMN product_description text;'%(tablename)


In [52]:
tmp = pypetl.fromDBSecret('rds', "SELECT * FROM public.%s LIMIT 5;"%(tablename_source),
    cache=True
)
for x in tmp.fieldnames():
    print(x)
    print(tmp.typeset(x))

status_id
set()
status_name
set()
date_in
set()
user_in
set()
date_up
set()
user_up
set()
status_record
set()


In [53]:
tmp = pypetl.fromDBSecret('rds', "SELECT * FROM public.%s LIMIT 5;"%(tablename_source),
    cache=True
)
print(tmp)

+-----------+-------------+---------+---------+---------+---------+---------------+
| status_id | status_name | date_in | user_in | date_up | user_up | status_record |
+===========+=============+=========+=========+=========+=========+===============+

